In [50]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re

# === НАСТРОЙКИ ===
KEYWORDS = ['трамп', 'канад', 'германи', 'сша', 'австрали', 'япони', 'великобритани', 'китай', 'росси']
DAYS_BACK = 2
URL = 'https://www.rbc.ru/economics/'

# === ЗАПУСК SELENIUM ===
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

print("[INFO] Загружаем страницу...")
driver.get(URL)
time.sleep(5)  # Лучше заменить на явные ожидания для продакшена

html = driver.page_source
driver.quit()

# === ПАРСИНГ И ОДНОВРЕМЕННАЯ ФИЛЬТРАЦИЯ ===
soup = BeautifulSoup(html, 'html.parser')
items = soup.find_all('div', class_='item')
print(f"[INFO] Найдено карточек на странице: {len(items)}")

now = datetime.now()
filtered_articles = []

for item in items:
    title_tag = item.find('span', class_='item__title')
    link_tag = item.find('a', class_='item__link')

    if not title_tag or not link_tag:
        continue

    title = ''.join(title_tag.stripped_strings)
    link = link_tag['href']

    # Извлекаем дату из URL
    match = re.search(r'/(\d{2})/(\d{2})/(\d{4})/', link)
    if not match:
        continue
    day, month, year = map(int, match.groups())
    pub_date = datetime(year, month, day)

    # Фильтрация по дате — пропускаем новости старше DAYS_BACK
    if (now - pub_date).days > DAYS_BACK:
        continue

    # Фильтрация по ключевым словам — пропускаем, если не содержат ни одного ключевого слова
    title_lower = title.lower()
    if not any(kw in title_lower for kw in KEYWORDS):
        continue

    filtered_articles.append({
        'title': title,
        'link': link,
        'date': pub_date
    })

# === ВЫВОД РЕЗУЛЬТАТОВ ===
if not filtered_articles:
    print("\n❌ Нет подходящих новостей.")
else:
    print(f"\n✅ Найдено {len(filtered_articles)} новостей по фильтру:\n")
    for i, a in enumerate(filtered_articles, 1):
        print(f"[{i}] [{a['date'].strftime('%Y-%m-%d')}] {a['title']}\n     🔗 {a['link']}\n")


[INFO] Загружаем страницу...
[INFO] Найдено карточек на странице: 20

✅ Найдено 3 новостей по фильтру:

[1] [2025-08-10] Поставки лекарств из США в Россию за год выросли почти в 7 раз
     🔗 https://www.rbc.ru/economics/10/08/2025/6898bc319a7947e41c8a4b05

[2] [2025-08-10] FT узнала требование Китая в торговых переговорах с США
     🔗 https://www.rbc.ru/economics/10/08/2025/68982c5e9a79477e692f4854

[3] [2025-08-09] Бразилия испугалась стать новой мишенью Трампа из-за российского топлива
     🔗 https://www.rbc.ru/economics/09/08/2025/689743b69a794797f8463b44



In [49]:
import re
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
from IPython.display import display, Markdown

KEYWORDS = ['трамп', 'trump', 'дональд']
DAYS_BACK = 2
URL = 'https://iz.ru/news'
MAX_NEWS = 50
BASE_URL = 'https://iz.ru'

def extract_json_from_html(html):
    start_pattern = r'window\.recommendationBlockList\s*=\s*{'
    start_match = re.search(start_pattern, html)
    if not start_match:
        return None

    start_index = start_match.end() - 1  # позиция первой {

    braces = 0
    end_index = start_index
    for i, ch in enumerate(html[start_index:], start=start_index):
        if ch == '{':
            braces += 1
        elif ch == '}':
            braces -= 1
            if braces == 0:
                end_index = i
                break

    json_text = html[start_index:end_index+1]
    return json_text

def parse_articles(json_data):
    articles = []
    for key in ['even', 'odd']:
        for item in json_data.get(key, []):
            path = item.get('path', '')
            if path.startswith('http://') or path.startswith('https://'):
                full_link = path
            else:
                full_link = BASE_URL + path if path.startswith('/') else BASE_URL + '/' + path
            articles.append({
                'title': item.get('title', ''),
                'link': full_link,
                'date': None,
                'reference': item.get('reference', '')
            })
    return articles

def main():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(URL)

    html = driver.page_source
    driver.quit()

    json_text = extract_json_from_html(html)
    if not json_text:
        print("❌ Не удалось найти данные новостей в HTML")
        return

    data = json.loads(json_text)
    articles = parse_articles(data)

    filtered = []
    now = datetime.now()
    cutoff = now - timedelta(days=DAYS_BACK)

    for a in articles:
        title_lower = a['title'].lower()
        if not any(kw in title_lower for kw in KEYWORDS):
            continue
        filtered.append(a)
        if len(filtered) >= MAX_NEWS:
            break

    if not filtered:
        print("❌ Нет подходящих новостей.")
        return

    print(f"✅ Найдено {len(filtered)} новостей по фильтру:\n")
    for i, a in enumerate(filtered, 1):
        # Вывод кликаемой ссылки через Markdown
        display(Markdown(f"**[{i}] {a['title']}**  \n🔗 [{a['link']}]({a['link']})\n"))

if __name__ == '__main__':
    main()


✅ Найдено 4 новостей по фильтру:



**[1] Дональд Трамп заявил о сдаче когнитивного теста на отлично**  
🔗 [https://iz.ru/1935030/2025-08-11/donald-tramp-zaiavil-o-sdache-kognitivnogo-testa-na-otlichno](https://iz.ru/1935030/2025-08-11/donald-tramp-zaiavil-o-sdache-kognitivnogo-testa-na-otlichno)


**[2] США и Европа готовятся к встрече Трампа с Путиным на Аляске. Что пишут СМИ**  
🔗 [https://iz.ru/1934859/2025-08-11/ssha-i-evropa-gotoviatsia-k-vstreche-trampa-s-putinym-na-aliaske-chto-pishut-smi](https://iz.ru/1934859/2025-08-11/ssha-i-evropa-gotoviatsia-k-vstreche-trampa-s-putinym-na-aliaske-chto-pishut-smi)


**[3] Дональд Трамп заявил о сдаче когнитивного теста на отлично**  
🔗 [https://iz.ru/1935030/2025-08-11/donald-tramp-zaiavil-o-sdache-kognitivnogo-testa-na-otlichno](https://iz.ru/1935030/2025-08-11/donald-tramp-zaiavil-o-sdache-kognitivnogo-testa-na-otlichno)


**[4] США и Европа готовятся к встрече Трампа с Путиным на Аляске. Что пишут СМИ**  
🔗 [https://iz.ru/1934859/2025-08-11/ssha-i-evropa-gotoviatsia-k-vstreche-trampa-s-putinym-na-aliaske-chto-pishut-smi](https://iz.ru/1934859/2025-08-11/ssha-i-evropa-gotoviatsia-k-vstreche-trampa-s-putinym-na-aliaske-chto-pishut-smi)


In [51]:
import re
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import time

KEYWORDS = ['трамп', 'trump', 'дональд', 'канад', 'германи', 'сша', 'австрали', 'япони', 'великобритани', 'китай', 'росси']
DAYS_BACK = 2
MAX_NEWS = 50

def parse_rbc(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='item')
    now = datetime.now()
    filtered_articles = []

    for item in items:
        title_tag = item.find('span', class_='item__title')
        link_tag = item.find('a', class_='item__link')

        if not title_tag or not link_tag:
            continue

        title = ''.join(title_tag.stripped_strings)
        link = link_tag['href']

        match = re.search(r'/(\d{2})/(\d{2})/(\d{4})/', link)
        if not match:
            continue
        day, month, year = map(int, match.groups())
        pub_date = datetime(year, month, day)

        if (now - pub_date).days > DAYS_BACK:
            continue

        title_lower = title.lower()
        if not any(kw in title_lower for kw in KEYWORDS):
            continue

        filtered_articles.append({
            'title': title,
            'link': link,
            'date': pub_date
        })

    return filtered_articles

def extract_json_from_html(html):
    start_pattern = r'window\.recommendationBlockList\s*=\s*{'
    start_match = re.search(start_pattern, html)
    if not start_match:
        return None

    start_index = start_match.end() - 1
    braces = 0
    end_index = start_index
    for i, ch in enumerate(html[start_index:], start=start_index):
        if ch == '{':
            braces += 1
        elif ch == '}':
            braces -= 1
            if braces == 0:
                end_index = i
                break

    return html[start_index:end_index+1]

def parse_iz(json_data):
    BASE_URL = 'https://iz.ru'
    articles = []
    for key in ['even', 'odd']:
        for item in json_data.get(key, []):
            path = item.get('path', '')
            if path.startswith('http://') or path.startswith('https://'):
                full_link = path
            else:
                full_link = BASE_URL + path if path.startswith('/') else BASE_URL + '/' + path
            articles.append({
                'title': item.get('title', ''),
                'link': full_link,
                'date': None,
            })
    return articles

def fetch_with_selenium(url):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(5)  # можно заменить на явные ожидания
    html = driver.page_source
    driver.quit()
    return html

def main():
    # --- RBC ---
    print("[INFO] Загружаем RBC...")
    rbc_html = fetch_with_selenium('https://www.rbc.ru/economics/')
    rbc_articles = parse_rbc(rbc_html)

    # --- Iz.ru ---
    print("[INFO] Загружаем Iz.ru...")
    iz_html = fetch_with_selenium('https://iz.ru/news')
    json_text = extract_json_from_html(iz_html)
    if not json_text:
        print("❌ Не удалось найти новости Iz.ru")
        iz_articles = []
    else:
        data = json.loads(json_text)
        iz_articles = parse_iz(data)

    # Объединяем статьи
    all_articles = rbc_articles + iz_articles

    # Фильтруем по ключевым словам и дате для Iz.ru (даты нет, пропускаем фильтр по времени)
    now = datetime.now()
    cutoff = now - timedelta(days=DAYS_BACK)

    filtered = []
    for a in all_articles:
        title_lower = a['title'].lower()
        if not any(kw in title_lower for kw in KEYWORDS):
            continue
        # Для RBC проверка даты уже сделана
        # Для Iz.ru даты нет, считаем допустимыми все
        filtered.append(a)
        if len(filtered) >= MAX_NEWS:
            break

    if not filtered:
        print("❌ Нет подходящих новостей.")
        return

    print(f"✅ Найдено {len(filtered)} новостей по фильтру:\n")
    for i, a in enumerate(filtered, 1):
        # В Jupyter делаем кликабельный вывод
        display(Markdown(f"**[{i}] {a['title']}**  \n🔗 [{a['link']}]({a['link']})\n"))

if __name__ == '__main__':
    main()


[INFO] Загружаем RBC...
[INFO] Загружаем Iz.ru...
✅ Найдено 13 новостей по фильтру:



**[1] Поставки лекарств из США в Россию за год выросли почти в 7 раз**  
🔗 [https://www.rbc.ru/economics/10/08/2025/6898bc319a7947e41c8a4b05](https://www.rbc.ru/economics/10/08/2025/6898bc319a7947e41c8a4b05)


**[2] FT узнала требование Китая в торговых переговорах с США**  
🔗 [https://www.rbc.ru/economics/10/08/2025/68982c5e9a79477e692f4854](https://www.rbc.ru/economics/10/08/2025/68982c5e9a79477e692f4854)


**[3] Бразилия испугалась стать новой мишенью Трампа из-за российского топлива**  
🔗 [https://www.rbc.ru/economics/09/08/2025/689743b69a794797f8463b44](https://www.rbc.ru/economics/09/08/2025/689743b69a794797f8463b44)


**[4] Дональд Трамп заявил о сдаче когнитивного теста на отлично**  
🔗 [https://iz.ru/1935030/2025-08-11/donald-tramp-zaiavil-o-sdache-kognitivnogo-testa-na-otlichno](https://iz.ru/1935030/2025-08-11/donald-tramp-zaiavil-o-sdache-kognitivnogo-testa-na-otlichno)


...

**[9] Дональд Трамп заявил о сдаче когнитивного теста на отлично**  
🔗 [https://iz.ru/1935030/2025-08-11/donald-tramp-zaiavil-o-sdache-kognitivnogo-testa-na-otlichno](https://iz.ru/1935030/2025-08-11/donald-tramp-zaiavil-o-sdache-kognitivnogo-testa-na-otlichno)


**[10] США и Европа готовятся к встрече Трампа с Путиным на Аляске. Что пишут СМИ**  
🔗 [https://iz.ru/1934859/2025-08-11/ssha-i-evropa-gotoviatsia-k-vstreche-trampa-s-putinym-na-aliaske-chto-pishut-smi](https://iz.ru/1934859/2025-08-11/ssha-i-evropa-gotoviatsia-k-vstreche-trampa-s-putinym-na-aliaske-chto-pishut-smi)


**[11] Нарезка округов и «бюллетени-бабочки». Как в США манипулируют выборами**  
🔗 [https://iz.ru/1932577/2025-08-11/narezka-okrugov-i-bulleteni-babocki-kak-v-ssa-manipuliruut-vyborami](https://iz.ru/1932577/2025-08-11/narezka-okrugov-i-bulleteni-babocki-kak-v-ssa-manipuliruut-vyborami)


**[12] Пилот NASCAR Брюэр умер от сердечного приступа во время гонки в США**  
🔗 [https://iz.ru/](https://iz.ru/)


**[13] Как в России развивают речные и морские магистрали**  
🔗 [https://iz.ru/](https://iz.ru/)


In [54]:
import re
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from IPython.display import display, Markdown  # для кликабельных ссылок в Jupyter, если не используешь — заменить на print

# === НАСТРОЙКИ ===
KEYWORDS = ['трамп', 'trump', 'дональд', 'канад', 'германи', 'сша', 'австрали', 'япони', 'великобритани', 'китай', 'росси']
DAYS_BACK = 2
MAX_NEWS_IZ = 50

# --- URLs ---
RBC_URL = 'https://www.rbc.ru/economics/'
IZ_URL = 'https://iz.ru/news'
IZ_BASE_URL = 'https://iz.ru'

def fetch_rbc_articles(driver, now):
    driver.get(RBC_URL)
    driver.implicitly_wait(5)  # лучше использовать явные ожидания, но для простоты
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='item')
    articles = []

    for item in items:
        title_tag = item.find('span', class_='item__title')
        link_tag = item.find('a', class_='item__link')

        if not title_tag or not link_tag:
            continue

        title = ''.join(title_tag.stripped_strings)
        link = link_tag['href']

        # Извлекаем дату из URL
        match = re.search(r'/(\d{2})/(\d{2})/(\d{4})/', link)
        if not match:
            continue
        day, month, year = map(int, match.groups())
        pub_date = datetime(year, month, day)

        # Фильтр по дате
        if (now - pub_date).days > DAYS_BACK:
            continue

        # Фильтр по ключевым словам
        if not any(kw in title.lower() for kw in KEYWORDS):
            continue

        articles.append({
            'title': title,
            'link': link,
            'date': pub_date,
        })

    return articles

def extract_json_from_html(html):
    start_pattern = r'window\.recommendationBlockList\s*=\s*{'
    start_match = re.search(start_pattern, html)
    if not start_match:
        return None

    start_index = start_match.end() - 1
    braces = 0
    end_index = start_index
    for i, ch in enumerate(html[start_index:], start=start_index):
        if ch == '{':
            braces += 1
        elif ch == '}':
            braces -= 1
            if braces == 0:
                end_index = i
                break
    json_text = html[start_index:end_index+1]
    return json_text

def parse_iz(json_data):
    articles_dict = {}
    for key in ['even', 'odd']:
        for item in json_data.get(key, []):
            article_id = item.get('id') or item.get('reference') or item.get('path')
            path = item.get('path', '').strip()
            if not path:
                path = item.get('reference', '').strip()

            # Проверка валидности ссылки
            if not path or path in ['/', IZ_BASE_URL, IZ_BASE_URL + '/']:
                print(f"[WARN] Пропущена новость без валидной ссылки: Заголовок: \"{item.get('title', '')}\"")
                continue

            if path.startswith('http://') or path.startswith('https://'):
                full_link = path
            else:
                full_link = IZ_BASE_URL + path if path.startswith('/') else IZ_BASE_URL + '/' + path

            unique_key = article_id if article_id else full_link

            if unique_key in articles_dict:
                continue

            articles_dict[unique_key] = {
                'title': item.get('title', ''),
                'link': full_link,
                'date': None,  # в данных iz.ru даты нет, можно потом пытаться парсить с сайта, но это сложнее
            }

    return list(articles_dict.values())

def fetch_iz_articles(driver, now):
    driver.get(IZ_URL)
    driver.implicitly_wait(5)
    html = driver.page_source

    json_text = extract_json_from_html(html)
    if not json_text:
        print("❌ Не удалось найти данные новостей iz.ru в HTML")
        return []

    data = json.loads(json_text)
    articles = parse_iz(data)

    filtered = []
    for a in articles:
        if not any(kw in a['title'].lower() for kw in KEYWORDS):
            continue
        filtered.append(a)
        if len(filtered) >= MAX_NEWS_IZ:
            break

    return filtered

def print_articles(articles):
    if not articles:
        print("❌ Нет подходящих новостей.")
        return
    print(f"✅ Найдено {len(articles)} новостей по фильтру:\n")
    for i, a in enumerate(articles, 1):
        # Кликабельные ссылки, если запускаешь в Jupyter/IPython
        try:
            display(Markdown(f"**[{i}] {a['title']}**  \n🔗 [{a['link']}]({a['link']})\n"))
        except NameError:
            # Если display или Markdown не доступны — простой вывод
            print(f"[{i}] {a['title']}\n🔗 {a['link']}\n")

def main():
    now = datetime.now()

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=chrome_options)

    # Парсим RBC
    rbc_articles = fetch_rbc_articles(driver, now)

    # Парсим Iz.ru
    iz_articles = fetch_iz_articles(driver, now)

    driver.quit()

    # Объединяем результаты
    all_articles = rbc_articles + iz_articles

    # Сортируем по дате — у iz.ru даты нет, поэтому они идут после RBC
    all_articles.sort(key=lambda x: x['date'] if x['date'] else datetime.min, reverse=True)

    print_articles(all_articles)

if __name__ == '__main__':
    main()


✅ Найдено 8 новостей по фильтру:



**[1] Поставки лекарств из США в Россию за год выросли почти в 7 раз**  
🔗 [https://www.rbc.ru/economics/10/08/2025/6898bc319a7947e41c8a4b05](https://www.rbc.ru/economics/10/08/2025/6898bc319a7947e41c8a4b05)


**[2] FT узнала требование Китая в торговых переговорах с США**  
🔗 [https://www.rbc.ru/economics/10/08/2025/68982c5e9a79477e692f4854](https://www.rbc.ru/economics/10/08/2025/68982c5e9a79477e692f4854)


**[3] Бразилия испугалась стать новой мишенью Трампа из-за российского топлива**  
🔗 [https://www.rbc.ru/economics/09/08/2025/689743b69a794797f8463b44](https://www.rbc.ru/economics/09/08/2025/689743b69a794797f8463b44)


**[4] Дональд Трамп заявил о сдаче когнитивного теста на отлично**  
🔗 [https://iz.ru/1935030/2025-08-11/donald-tramp-zaiavil-o-sdache-kognitivnogo-testa-na-otlichno](https://iz.ru/1935030/2025-08-11/donald-tramp-zaiavil-o-sdache-kognitivnogo-testa-na-otlichno)


**[5] США и Европа готовятся к встрече Трампа с Путиным на Аляске. Что пишут СМИ**  
🔗 [https://iz.ru/1934859/2025-08-11/ssha-i-evropa-gotoviatsia-k-vstreche-trampa-s-putinym-na-aliaske-chto-pishut-smi](https://iz.ru/1934859/2025-08-11/ssha-i-evropa-gotoviatsia-k-vstreche-trampa-s-putinym-na-aliaske-chto-pishut-smi)


**[6] Нарезка округов и «бюллетени-бабочки». Как в США манипулируют выборами**  
🔗 [https://iz.ru/1932577/2025-08-11/narezka-okrugov-i-bulleteni-babocki-kak-v-ssa-manipuliruut-vyborami](https://iz.ru/1932577/2025-08-11/narezka-okrugov-i-bulleteni-babocki-kak-v-ssa-manipuliruut-vyborami)


**[7] Овечкин сомневается, что снова сыграет за сборную России после отстранения**  
🔗 [https://www.sport-express.net/hockey/nhl/reviews/aleksandr-ovechkin-o-sbornoy-rossii-i-olimpiade-momentah-karery-i-seme-sovety-yunym-hokkeistam-i-sebe-2349633/](https://www.sport-express.net/hockey/nhl/reviews/aleksandr-ovechkin-o-sbornoy-rossii-i-olimpiade-momentah-karery-i-seme-sovety-yunym-hokkeistam-i-sebe-2349633/)


**[8] Как в России развивают речные и морские магистрали**  
🔗 [https://regnum.ru/news/3974432](https://regnum.ru/news/3974432)


In [57]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import re

KEYWORDS = ['экономика', 'рынок', 'инфляция', 'курс', 'банк', 'цена', 'снижение', 'рост', 'трамп', 'trump', 'дональд']
DAYS_BACK = 2
URL = 'https://tass.ru/ekonomika'
TARGET_NEWS_COUNT = 100
SCROLL_PAUSE_SEC = 2  # пауза после скролла для загрузки

def scroll_to_load(driver, target_count):
    last_height = driver.execute_script("return document.body.scrollHeight")
    news_count = 0
    tries = 0
    max_tries = 20  # чтобы не зациклиться

    while news_count < target_count and tries < max_tries:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)

        # Проверяем сколько карточек сейчас на странице
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.find_all('article', class_=re.compile(r'list-item|card-mini'))
        news_count = len(articles)
        if news_count >= target_count:
            break

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            tries += 1
        else:
            tries = 0
            last_height = new_height

def parse_tass_news():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(180)

    print("[INFO] Загружаем страницу TASS...")
    try:
        driver.get(URL)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, 'article'))
        )
        scroll_to_load(driver, TARGET_NEWS_COUNT)
    except Exception as e:
        print(f"[ERROR] Ошибка при загрузке страницы: {e}")
        driver.quit()
        return

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find_all('article', class_=re.compile(r'list-item|card-mini'))
    print(f"[INFO] Всего найдено новостей после скролла: {len(articles)}")

    now = datetime.now()
    cutoff_date = now - timedelta(days=DAYS_BACK)

    filtered_articles = []

    for article in articles:
        title_tag = article.find('a', class_='list-item__title') or article.find('a', class_='card-mini__title')
        if not title_tag:
            continue
        title = title_tag.get_text(strip=True)
        link = title_tag['href']

        time_tag = article.find('time')
        if time_tag and time_tag.has_attr('datetime'):
            pub_date_str = time_tag['datetime']
            try:
                pub_date = datetime.fromisoformat(pub_date_str.replace('Z', '+00:00')).replace(tzinfo=None)
            except Exception:
                continue
        else:
            continue

        if pub_date < cutoff_date:
            continue

        title_lower = title.lower()
        if not any(kw in title_lower for kw in KEYWORDS):
            continue

        if link.startswith('/'):
            link = 'https://tass.ru' + link

        filtered_articles.append({
            'title': title,
            'link': link,
            'date': pub_date
        })

        if len(filtered_articles) >= TARGET_NEWS_COUNT:
            break

    if not filtered_articles:
        print("❌ Нет подходящих новостей.")
        return

    print(f"\n✅ Найдено {len(filtered_articles)} новостей по фильтру за последние {DAYS_BACK} дня:\n")
    for i, a in enumerate(filtered_articles, 1):
        print(f"[{i}] [{a['date'].strftime('%Y-%m-%d %H:%M')}] {a['title']}\n     🔗 {a['link']}\n")

if __name__ == "__main__":
    parse_tass_news()


[INFO] Загружаем страницу TASS...
[ERROR] Ошибка при загрузке страницы: Message: 
Stacktrace:
0   chromedriver                        0x00000001005c4b38 cxxbridge1$str$ptr + 2722088
1   chromedriver                        0x00000001005bcaa8 cxxbridge1$str$ptr + 2689176
2   chromedriver                        0x000000010010e33c cxxbridge1$string$len + 90648
3   chromedriver                        0x0000000100155494 cxxbridge1$string$len + 381808
4   chromedriver                        0x0000000100196880 cxxbridge1$string$len + 649052
5   chromedriver                        0x0000000100149784 cxxbridge1$string$len + 333408
6   chromedriver                        0x0000000100587eb4 cxxbridge1$str$ptr + 2473124
7   chromedriver                        0x000000010058b120 cxxbridge1$str$ptr + 2486032
8   chromedriver                        0x00000001005696b4 cxxbridge1$str$ptr + 2348196
9   chromedriver                        0x000000010058b9dc cxxbridge1$str$ptr + 2488268
10  chromedriver   